In [3]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')

############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

F:\github\narrative_conservatism\code


In [2]:
########################################################################################
############ Merge IBES quarterly data with CRSP_COMP_EDGAR data #######################
########################################################################################

########### Read IBES raw data file
ibes_cols = ['cusip8', 'cname', 'fpi', 'value', 'fpedats', 'anndats', 'actual', 'anndats_act']
ibes = pd.read_csv('..\\filings\\ibes.csv', usecols = ibes_cols)
print('number of cusip-fpedats-analyst: ' + str(ibes.shape[0]))

### Reorder and rename IBES columns
ibes = ibes[['cusip8', 'cname', 'fpedats', 'value', 'actual', 'anndats_act', 'anndats', 'fpi']]
ibes = ibes.rename(columns={'cusip8': 'cusip'})

### correct fpedats format
ibes['fpedats'] = ibes['fpedats'].str.replace('jan','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('feb','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('mar','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('apr','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('may','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('jun','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('jul','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('aug','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('sep','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('oct','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('nov','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('dec','12')
ibes['fpedats'] = pd.to_datetime(ibes['fpedats'], format='%d%m%Y')

print(ibes['fpedats'].describe())

### Delete missing cusip8, actual
del_cusip = ibes[ibes['cusip'].isnull()].shape[0]
ibes = ibes[ibes['cusip'].isnull() == False]
print('number of obs. that contain missing cusip: ' + str(del_cusip))

del_actual = ibes[ibes['actual'].isnull()].shape[0]
ibes = ibes[ibes['actual'].isnull() == False]
print('number of obs. that contain missing actual: ' + str(del_actual))

### aggregate by cusip-fpedats and get actual, median, afe and consensus from ibes raw data
ibes_css = ibes.groupby(['cusip', 'fpedats'])['actual'].median().to_frame()
ibes_css['median'] = ibes.groupby(['cusip', 'fpedats'])['value'].median().to_frame()
ibes_css['afe'] = ibes_css['actual'] - ibes_css['median']
ibes_css['consensus'] = ibes.groupby(['cusip', 'fpedats'])['value'].mean().to_frame()
ibes_css['leap_consensus'] = ibes_css.groupby(['cusip'])['consensus'].shift(-1)

### make a consensus forecast dataset by cusip-fpedats
ibes = ibes.loc[ibes.duplicated(subset=['cusip', 'fpedats']) == False, ['cusip','fpedats']]
ibes = ibes.join(ibes_css, on=['cusip', 'fpedats'])

### create merge date_key - %y only
ibes['date_key'] = ibes['fpedats'].astype(str).str[:-6].astype(int)

print('number of cusip-fpedats, dropping missing cusip and actual: ' + str(ibes.shape[0]))

number of cusip-fpedats-analyst: 9812071
count                 9812071
unique                    446
top       2015-12-31 00:00:00
freq                   426980
first     1981-12-31 00:00:00
last      2019-07-31 00:00:00
Name: fpedats, dtype: object
number of obs. that contain missing cusip: 60781
number of obs. that contain missing actual: 1305946
number of cusip-fpedats, dropping missing cusip and actual: 155539


In [6]:
##################### Read EDGAR_CRSP_COMP and creat merge date_key
crsp_comp_edgar = pd.read_csv('..\\filings\\id_crsp_comp_text_10-Q.csv')
crsp_comp_edgar['date_key'] = crsp_comp_edgar['fyearq'].astype(int)

##################### inner merge EDGAR_CRSP_COMP and IBES, key not unique in both datasets
crsp_comp_edgar_ibes = pd.merge(crsp_comp_edgar, ibes, on = ['cusip', 'date_key'], how='inner', validate = 'm:m')
print('number of cusip-date after merging: ' + str(crsp_comp_edgar_ibes.shape[0]))

number of cusip-date after merging: 110109


In [7]:
########################################################################################
############ Merge CRSP_COMP_EDGAR_IBES data with SEG data #############################
########################################################################################

########### Read SEG raw data file
seg_cols = ['gvkey', 'stype', 'sid', 'datadate', 'snms', 'cusip', 'cik']
seg = pd.read_csv('..\\filings\\compustat_seg.csv', usecols = seg_cols)
print('number of cusip-fpedats-analyst: ' + str(seg.shape[0]))

### Reorder and rename IBES columns
seg = seg[['gvkey', 'cik', 'cusip', 'datadate', 'stype', 'sid', 'snms']]
seg = seg.rename(columns={'datadate': 'date_comp'})

### parse date_comp format
seg['date_comp'] = pd.to_datetime(seg['date_comp'], format='%Y%m%d')

### aggregate by gvkey-date_comp-stype and get count of sid
seg_count = seg.groupby(['gvkey', 'date_comp', 'stype'])['sid'].count().to_frame()

### make a segment dataset by gvkey-date_comp-stype
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp', 'stype']) == False, ['gvkey', 'date_comp', 'stype']]
seg = seg.join(seg_count, on=['gvkey', 'date_comp', 'stype'])
seg_bus = seg[seg['stype']=='BUSSEG']
seg_bus = seg_bus.drop(columns=['stype'])
seg_bus = seg_bus.rename(columns={'sid': 'nseg_bus'})
seg_geo = seg[seg['stype']=='GEOSEG']
seg_geo = seg_geo.drop(columns=['stype'])
seg_geo = seg_geo.rename(columns={'sid': 'nseg_geo'})
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp']) == False, ['gvkey', 'date_comp']]
seg = pd.merge(seg, seg_bus, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = pd.merge(seg, seg_geo, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = seg.sort_values(by=['gvkey', 'date_comp'])
seg.loc[seg['nseg_geo'].isnull(), 'nseg_geo'] = 1
seg.loc[seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
seg['nseg_bus'] = seg['nseg_bus'].astype(int)
seg['nseg_geo'] = seg['nseg_geo'].astype(int)

### create merge date_key - %y only
seg['date_key'] = seg['date_comp'].astype(str).str[:-6].astype(int)
seg = seg.drop(columns=['date_comp'])

print('number of gvkey-datadate in SEGMENT data: ' + str(seg.shape[0]))

number of cusip-fpedats-analyst: 452653
number of gvkey-datadate in SEGMENT data: 50876


In [8]:
##################### left merge EDGAR_CRSP_COMP_IBES and SEG, key not unique in both datasets
crsp_comp_edgar_ibes_seg_left = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='left', validate = 'm:m')
print('Number of gvkey-quarter obs. after merging, left: '+ str(crsp_comp_edgar_ibes_seg_left.shape[0]))
crsp_comp_edgar_ibes_seg_inner = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='inner', validate = 'm:m')
print('Number of gvkey-quarter obs. after merging, inner: '+ str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

##################### fill missing segments in crsp_comp_edgar_ibes_seg_left with 1
crsp_comp_edgar_ibes_seg_left.loc[crsp_comp_edgar_ibes_seg_left['nseg_bus'].isnull(), 'nseg_bus'] = 1
crsp_comp_edgar_ibes_seg_left.loc[crsp_comp_edgar_ibes_seg_left['nseg_geo'].isnull(), 'nseg_geo'] = 1

Number of gvkey-quarter obs. after merging, left: 110128
Number of gvkey-quarter obs. after merging, inner: 24282


In [9]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

################### CRSP_COMP_EDGAR_IBES_SEG_LEFT ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg_left['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['age'])
######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg_left['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['nseg_bus'])
crsp_comp_edgar_ibes_seg_left['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_left['nseg_geo'])
######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_left['AFE'] = crsp_comp_edgar_ibes_seg_left['afe']/crsp_comp_edgar_ibes_seg_left['prccq']
######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_left['AF'] = crsp_comp_edgar_ibes_seg_left['leap_consensus']/crsp_comp_edgar_ibes_seg_left['prccq']

################### CRSP_COMP_EDGAR_IBES_SEG_INNER ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg_inner['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['age'])
######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg_inner['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['nseg_bus'])
crsp_comp_edgar_ibes_seg_inner['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg_inner['nseg_geo'])
######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_inner['AFE'] = crsp_comp_edgar_ibes_seg_inner['afe']/crsp_comp_edgar_ibes_seg_inner['prccq']
######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg_inner['AF'] = crsp_comp_edgar_ibes_seg_inner['leap_consensus']/crsp_comp_edgar_ibes_seg_inner['prccq']

In [10]:
########################################################################################
########################## Variable Screening (LEFT) ###################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg_left[(crsp_comp_edgar_ibes_seg_left['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg_left['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[(crsp_comp_edgar_ibes_seg_left['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg_left['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_left = crsp_comp_edgar_ibes_seg_left[crsp_comp_edgar_ibes_seg_left['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg_left.shape[0]))

############## Winsorize SIZE, MTB, LEV
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

crsp_comp_edgar_ibes_seg_left = winsorize(crsp_comp_edgar_ibes_seg_left, ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET'])

############## Save merged ID_CRSP_COMP_TEXT to csv file
crsp_comp_edgar_ibes_seg_left.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

number of files that have positive/negative infinity DEARN: 1
number of obs. that contain missing AF: 15241
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 64
number of obs. that contain missing STD_EARN: 3162
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 91660


In [11]:
########################################################################################
########################## Variable Screening (INNER) ##################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg_inner[(crsp_comp_edgar_ibes_seg_inner['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg_inner['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[(crsp_comp_edgar_ibes_seg_inner['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg_inner['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg_inner = crsp_comp_edgar_ibes_seg_inner[crsp_comp_edgar_ibes_seg_inner['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

############## Winsorize SIZE, MTB, LEV
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

crsp_comp_edgar_ibes_seg_inner = winsorize(crsp_comp_edgar_ibes_seg_inner, ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET'])

############## Save merged ID_CRSP_COMP_TEXT to csv file
crsp_comp_edgar_ibes_seg_inner.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

number of files that have positive/negative infinity DEARN: 1
number of obs. that contain missing AF: 6177
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 7
number of obs. that contain missing STD_EARN: 541
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 17556


In [29]:
########################################################################################
############### Table 3: Summary Statistics and Correlation Matrix #####################
########################################################################################

############# Table 3 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_ibes_seg_left[['NW','nw', 'TONE','TLAG', \
                                               'RET', 'NEG', 'SIZE', 'MTB', 'LEV', \
                                               'AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', 'LOSS' \
                                             ]]

T3PA = selected_vars.describe().transpose() 

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_ibes_seg_left.describe().transpose()

############# Save T3PA
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T3PA.to_excel(writer, sheet_name='T3PA_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T3PA.to_excel(table_path, sheet_name='T3PA_raw', float_format="%.4f")

T3PA

,count,mean,std,min,25%,50%,75%,max
NW,91660.0,8.945742,0.763618,7.044033,8.423542,9.009631,9.477233,13.490002
nw,91660.0,10215.750633,9672.798080,1145.000000,4552.000000,8180.500000,13058.000000,722159.000000
TONE,91660.0,-8.456708,6.884101,-64.542890,-12.434190,-7.472527,-3.641540,22.287390
TLAG,91660.0,39.021962,6.248849,0.000000,36.000000,40.000000,44.000000,52.000000
RET,91660.0,0.018251,0.253128,-1.578704,-0.112629,0.007303,0.129897,4.849226
NEG,91660.0,0.482915,0.499711,0.000000,0.000000,0.000000,1.000000,1.000000
SIZE,91660.0,6.446919,1.776287,2.001955,5.175067,6.317049,7.563056,11.205299
MTB,91660.0,3.516835,4.010382,0.288279,1.485811,2.343883,3.903106,30.875464
LEV,91660.0,0.192395,0.182147,0.000000,0.010620,0.162323,0.315081,0.724226
AF,91660.0,0.043051,0.066428,-0.262007,0.022596,0.048512,0.073176,0.227485


In [30]:
# full_summary

In [31]:
############# Table 3 Panel B: Correlation matrix for selected variables
######### pearson correlation
T3PB_pearson = selected_vars.corr(method='pearson')

# T3PB_pearson

In [32]:
######### spearman correlation
T3PB_spearman = selected_vars.corr(method='spearman')

# T3PB_spearman

In [35]:
######### Combine two correlation matrices. right-up matrix: pearson; left-down matrix: spearman 
for row in list(range(0, len(T3PB_spearman.index))):
    T3PB_spearman.iloc[row, row+1:] = T3PB_pearson.iloc[row, row+1:]
    
##### Save T3PB
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T3PB_spearman.to_excel(writer, sheet_name='T3PB_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T3PB_spearman.to_excel(table_path, sheet_name='T3PB_raw', float_format="%.4f")

T3PB_spearman

,NW,nw,TONE,TLAG,RET,NEG,SIZE,MTB,LEV,AF,AFE,BUSSEG,GEOSEG,AGE,EARN,DEARN,STD_EARN,STD_RET,LOSS
NW,1.000000,0.792064,-0.461391,-0.195034,-0.007662,0.003273,0.257594,0.058513,0.036546,-0.067000,0.012128,0.238073,0.258311,-0.037616,-0.115103,0.004835,0.089649,-0.033667,0.127585
nw,1.000000,1.000000,-0.349895,-0.092634,-0.012143,0.009686,0.166470,0.047867,0.048593,-0.053469,0.005028,0.145273,0.161767,-0.056462,-0.085825,0.002336,0.066425,-0.014404,0.092150
TONE,-0.485876,-0.485876,1.000000,0.025342,0.020641,-0.021134,-0.069691,-0.016080,0.069076,0.069352,0.098067,-0.033579,-0.069577,0.053973,0.155792,0.000439,-0.143925,-0.081186,-0.214672
TLAG,-0.266151,-0.266151,0.029134,1.000000,-0.022446,0.034372,-0.330877,-0.021927,0.009312,-0.092333,-0.127284,-0.229513,-0.245821,-0.228335,-0.136595,-0.004667,0.121087,0.189202,0.145981
RET,-0.008006,-0.008006,0.028909,-0.032631,1.000000,-0.684134,-0.064358,-0.025487,0.001982,-0.018298,0.154624,-0.012359,-0.020952,0.001892,0.063370,0.035208,0.010689,0.265996,-0.067800
NEG,0.004388,0.004388,-0.024174,0.033040,-0.865520,1.000000,0.000069,0.012222,-0.001805,0.015280,-0.124164,0.000897,0.011961,-0.018139,-0.071317,-0.018704,0.015715,-0.117549,0.077825
SIZE,0.266372,0.266372,-0.052507,-0.332723,-0.024044,-0.001059,1.000000,0.233328,0.099688,0.077005,0.269536,0.197835,0.215086,0.344031,0.258691,-0.013832,-0.197709,-0.309929,-0.263359
MTB,0.048182,0.048182,0.037387,-0.041901,-0.054724,0.032886,0.381616,1.000000,0.045436,-0.156338,0.120160,0.005178,0.002707,-0.088792,-0.041815,0.019718,0.159063,0.036059,0.034854
LEV,0.014623,0.014623,0.074565,-0.000515,0.002977,-0.004024,0.142888,-0.111247,1.000000,0.167018,-0.068185,0.034771,-0.005149,0.100994,0.039378,0.039438,-0.124449,-0.072086,-0.058262
AF,-0.017026,-0.017026,0.060340,-0.124692,-0.086673,0.071694,0.025468,-0.299291,0.250903,1.000000,0.057050,0.068346,0.079480,0.201940,0.472101,0.014761,-0.256539,-0.145172,-0.409958
